In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import numpy as np
import pylab as plt

from astropy.io import fits
from natsort import natsorted
from astropy.time import Time

import sys
sys.path.append('/home/scratch/psalas/LASSI/lassi-analysis_v2')

from grid import regridXYZMasked
from parabolas import rotateData
from simulateSignal import zernikePoly
from zernikies import getZernikeCoeffs
from lassiAnalysis import maskLeicaData
from utils.utils import midPoint, dishLimits, gridLimits

In [27]:
n = 512
sigma = 3
nZern = 36
maskRadius = 48
guess = [60, 0, 0, -50, 0, 0]
signal_registration = True
sigma_clip_diff = False
kernel_size = 1
do_replace_nans = False
fixed_reference = False
reference_scan = None
ref_scan_file = None
signal_type = b"REF"
fitsDir = '/home/gbtdata/TLASSI_200315'
scanDir = '/home/scratch/psalas/LASSI/gpus/output/'
output = 'zFitDict_1r.pickle'

ref_scans = ['2020_03_16_02:21:06',
            '2020_03_16_02:54:59',
            '2020_03_16_03:19:50',
            '2020_03_16_03:44:25',
            '2020_03_16_04:09:06',
            '2020_03_16_04:33:33',
            '2020_03_16_04:58:03',
            '2020_03_16_05:22:24',
            '2020_03_16_05:47:09',
            '2020_03_16_06:11:20',
            '2020_03_16_06:35:42',
            '2020_03_16_07:00:20',
            '2020_03_16_07:24:39',
            '2020_03_16_07:48:47',
            '2020_03_16_08:12:50',
            '2020_03_16_08:37:03',
            '2020_03_16_09:01:35',
            ]

In [28]:
zFitDict = {}
zFitDict['header'] = {'scan0': ref_scans[0], 'scanf': ref_scans[-1], 
                      'nZern': nZern,
                      'signal_registration': signal_registration,
                      'sigma_clip_diff': sigma_clip_diff,
                      'kernel_size': kernel_size,
                      'do_replace_nans': do_replace_nans,
                      'guess': guess, 'maskRadius': maskRadius,
                      'fixed_reference': fixed_reference,
                      'reference_scan': reference_scan,
                      'reference_scan_file': ref_scan_file,
                      'signal_type': signal_type,
                      'sigma': sigma, 'scan_dir': scanDir}

In [30]:
%%time

kwargs = {"sigma":sigma}

for i,fn in enumerate(ref_scans[:-1]):
    
    print('File: {0}'.format(fn))
    
    ff = "{}/LASSI/{}.fits".format(fitsDir, fn)
    hdu = fits.open(ff)
    head = hdu[0].header
    scan = head['SCAN']

    refScanFile = "{}/{}.ptx.csv".format(scanDir, fn)
    sigScanFile = "{}/{}.ptx.csv".format(scanDir, ref_scans[i+1])
        
    print('Masking file: {}'.format(refScanFile))
    ref_data = maskLeicaData(refScanFile, n=n, guess=guess, radialMask=True, maskRadius=maskRadius, **kwargs)
            
    # Extract the data we will use.
    xr, yr, zr = ref_data['origMasked']
    xrr, yrr, zrr = ref_data['rotated']
    cr = ref_data['parabolaFitCoeffs']
    dzr = ref_data['fitResidual']
    print('RMS in reference scan residual: {}'.format(dzr.std()))

    print('Masking file: {}'.format(sigScanFile))
    sig_data = maskLeicaData(sigScanFile, n=n, guess=guess, radialMask=True, maskRadius=maskRadius, **kwargs)

    xs, ys, zs = sig_data['origMasked']
    xsr, ysr, zsr = sig_data['rotated']
    cs = sig_data['parabolaFitCoeffs']
    dzs = sig_data['fitResidual']
    
    if signal_registration:
        cs = cs
    else:
        cs = cr
    
    # The data has been rotated, but we haven't applied the shifts.
    xrr = xrr - cr[1]
    yrr = yrr - cr[2]
    zrr = zrr - cr[3]
    xsr = xsr - cs[1]
    ysr = ysr - cs[2]
    zsr = zsr - cs[3]

    xmin, xmax = gridLimits(xrr, xsr)
    ymin, ymax = gridLimits(yrr, ysr)

    xrrg, yrrg, zrrg = regridXYZMasked(xrr, yrr, dzr, n=n, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
    xsrg, ysrg, zsrg = regridXYZMasked(xsr, ysr, dzs, n=n, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)

    diff = zsrg - zrrg
    diff = diff[::-1]
    if sigma_clip_diff:
        print('Sigma clipping the surface deformation map.')
        diff = sigma_clip(diff)

    if do_replace_nans:
        print('Replacing NaN values with interpolated ones.')
        diff = replace_nans(xrrg, yrrg, diff, kernel, maskRadius)

    print('RMS in deformation map: {}'.format(diff.std()))

    fitlist = getZernikeCoeffs(diff.filled(0), nZern, norm='active-surface')

    zInIndx = 0
    zInCoef = 0.
    
    zInCoefAS = np.nan
    print('Signal scan should have a Z{} with a coefficient of {}'.format(zInIndx, zInCoef))

    zDiff = np.nan

    zDiffAS = (fitlist[zInIndx] - zInCoefAS)/zInCoefAS
    print('Difference between active surface and recovered: {} %'.format(zDiffAS*100.))

    coeffs = np.array(fitlist)

    x = xrrg
    y = yrrg
    zpoly = zernikePoly(x, y, midPoint(x), midPoint(y), coeffs, verbose=False)

    resd = diff - zpoly
    resd_std = resd.std()
    print('Corrected surface RMS: {}'.format(resd_std))

    zFitDict[scan] = {'input zernike': zInIndx,
                      'input zernike value': zInCoef,
                      'active surface zernike value': zInCoefAS,
                      'reference scan': refScan,
                      'recovered zernike': fitlist,
                      'recovered zernike difference': zDiff,
                      'recovered zernike difference AS': zDiffAS,
                      'corrected surface rms': resd_std,
                      'deformation map rms': diff.std(),
                      'deformation map': diff,
                      'deformation map x': x,
                      'deformation map y': y,
                     }

File: 2020_03_16_02:21:06
Masking file: /home/scratch/psalas/LASSI/gpus/output//2020_03_16_02:21:06.ptx.csv
RMS in reference scan residual: 0.001859710290061695
Masking file: /home/scratch/psalas/LASSI/gpus/output//2020_03_16_02:54:59.ptx.csv
RMS in deformation map: 0.0004548017342037257
Signal scan should have a Z0 with a coefficient of 0.0
Difference between active surface and recovered: nan %
Corrected surface RMS: 0.0004114672685709139
File: 2020_03_16_02:54:59
Masking file: /home/scratch/psalas/LASSI/gpus/output//2020_03_16_02:54:59.ptx.csv
RMS in reference scan residual: 0.0017964497388714092
Masking file: /home/scratch/psalas/LASSI/gpus/output//2020_03_16_03:19:50.ptx.csv
RMS in deformation map: 0.0006759475389646389
Signal scan should have a Z0 with a coefficient of 0.0
Difference between active surface and recovered: nan %
Corrected surface RMS: 0.0006460017814859041
File: 2020_03_16_03:19:50
Masking file: /home/scratch/psalas/LASSI/gpus/output//2020_03_16_03:19:50.ptx.csv
RMS

In [31]:
pickle.dump(zFitDict, open(output, "wb"))